<a href="https://colab.research.google.com/github/milanzongor/ntut_dl_273045_project1/blob/master/ntut_deep_learning_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("jojo")

jojo
